In [1]:
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
import time

In [2]:
# Load the oversampled_train_data and preprocessed_test_data from pickle files
with open('dataset/preprocessed_train_data.pkl', 'rb') as f:
    loaded_oversampled_train_data = pickle.load(f)

with open('dataset/preprocessed_test_data.pkl', 'rb') as f:
    loaded_preprocessed_test_data = pickle.load(f)

In [3]:
class WaferMapDataset(Dataset):
    def __init__(self, data_dict):
        self.data_dict = data_dict
        self.samples = []
        self.labels = []
        self.label_map = {'Center': 0, 'Donut': 1, 'Edge-Loc': 2, 'Edge-Ring': 3, 'Loc': 4,
                          'Random': 5, 'Scratch': 6, 'Near-full': 7, 'none': 8}

        for label, images in self.data_dict.items():
            for image in images:
                self.samples.append(image)
                self.labels.append(label)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        # Convert the image to a PyTorch tensor and add a channel dimension
        image_tensor = torch.tensor(self.samples[idx], dtype=torch.float32).unsqueeze(0)
        label = self.label_map[self.labels[idx]]
        label_tensor  = torch.tensor(label, dtype=torch.long).squeeze()
        return image_tensor, label_tensor


In [4]:
# Create the custom datasets
train_dataset = WaferMapDataset(loaded_oversampled_train_data)
test_dataset = WaferMapDataset(loaded_preprocessed_test_data)

print(loaded_preprocessed_test_data.keys())
# Create the DataLoader instances
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

dict_keys(['Center', 'Donut', 'Edge-Loc', 'Edge-Ring', 'Loc', 'Random', 'Scratch', 'Near-full', 'none'])


In [5]:
def train(model, dataloader, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(dataloader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = model.compute_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    epoch_loss = running_loss / total
    accuracy = correct / total
    return epoch_loss, accuracy

In [6]:
def evaluate(model, dataloader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = model.compute_loss(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    epoch_loss = running_loss / total
    accuracy = correct / total
    return epoch_loss, accuracy

In [7]:
#Import Models
from models.CNN import cnn
from models.ResNet18 import CustomResNet18
from models.ResNet152 import CustomResNet152
from models.ResNet101 import CustomResNet101
from models.ResNet50 import CustomResNet50
from models.DCNN import OptResDCNN
from models.DenseNet121 import CustomDenseNet121
from models.MobileNet import CustomMobileNet

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Instantiate the model and move it to the appropriate device
models = [cnn(num_classes=9, dropout_rate=0).to(device),
          CustomResNet18(num_classes=9, dropout_rate=0).to(device),
          CustomResNet50(num_classes=9, dropout_rate=0).to(device),
          CustomResNet152(num_classes=9, dropout_rate=0).to(device),
          CustomResNet101(num_classes=9, dropout_rate=0).to(device),
          OptResDCNN(num_classes=9, dropout_rate=0).to(device),
          CustomDenseNet121(num_classes=9, dropout_rate=0).to(device),
          CustomMobileNet(num_classes=9, dropout_rate=0).to(device)]

#models = [CustomMobileNet(num_classes=9, dropout_rate=0).to(device)]

for model in models:
    model_name = type(model).__name__
    print(model_name)

    # Create an optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

    # Train and evaluate the model
    time_per_epoch = []
    loss_values = [[],[]]
    acc = [[],[]]
    num_epochs = 50
    test = 0.4
    for epoch in range(num_epochs):
        start_time = time.time()
        train_loss, train_accuracy = train(model, train_dataloader, optimizer, device)
        test_loss, test_accuracy = evaluate(model, test_dataloader, device)
        end_time = time.time()

        loss_values[0].append(train_loss)
        acc[0].append(train_accuracy)
        loss_values[1].append(test_loss)
        acc[1].append(test_accuracy)
        epoch_time = end_time - start_time
        time_per_epoch.append(epoch_time)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


    with open('results/'+model_name+'.pkl', 'wb') as file:
        pickle.dump({"time_per_epoch": time_per_epoch, "loss_values": loss_values, "accuracy": acc}, file)

C:\Users\xalzh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\xalzh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\xalzh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments o

CustomMobileNet


Evaluating: 100%|██████████| 24/24 [00:00<00:00, 122.19it/s]


Epoch 1/50
Train Loss: 0.0009, Train Accuracy: 0.9185
Test Loss: 0.0060, Test Accuracy: 0.5378


Evaluating: 100%|██████████| 24/24 [00:00<00:00, 160.00it/s]


...

Epoch 48/50
Train Loss: 0.0001, Train Accuracy: 0.9921
Test Loss: 0.0054, Test Accuracy: 0.6757


Evaluating: 100%|██████████| 24/24 [00:00<00:00, 157.90it/s]


Epoch 49/50
Train Loss: 0.0001, Train Accuracy: 0.9906
Test Loss: 0.0082, Test Accuracy: 0.5889


Evaluating: 100%|██████████| 24/24 [00:00<00:00, 158.94it/s]

Epoch 50/50
Train Loss: 0.0001, Train Accuracy: 0.9912
Test Loss: 0.0057, Test Accuracy: 0.6661
